###LSTM

In [1]:
import psutil
print(f"Memoria disponible: {psutil.virtual_memory().available / 1e9} GB")

Memoria disponible: 12.333109248 GB


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
#data = pd.read_csv("Datos_ConT.csv")
#data = pd.read_csv("Datos_Con5TResampled.csv")
#data = pd.read_csv("Datos_Con10TResampled.csv")
#data = pd.read_csv("Datos_Con30TResampled.csv")
data = pd.read_csv("Datos_Con1HResampled.csv")
#data = pd.read_csv("Datos_Con2HResampled.csv")

In [4]:
data.head()

,DateTime_,ErrTime,RMS,Az,Depth,ErrDepth,Ndef,Gap,mdist,Mdist_,Magnitud
0,2017-10-07 10:00:00,2.18,0.19,81.0,21.6,12.7,9.0,189.0,0.02,0.20,1.6
1,2017-10-07 11:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
2,2017-10-07 12:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
3,2017-10-07 13:00:00,2.19,0.20,67.0,28.2,13.0,8.0,149.0,0.03,0.13,2.7
4,2017-10-07 14:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


In [5]:
# Mover la columna "Magnitud" a la posición 0
magnitud_column = data['Magnitud']
data = data.drop('Magnitud', axis=1)
data.insert(0, 'Magnitud', magnitud_column)


In [6]:
data.head()

,Magnitud,DateTime_,ErrTime,RMS,Az,Depth,ErrDepth,Ndef,Gap,mdist,Mdist_
0,1.6,2017-10-07 10:00:00,2.18,0.19,81.0,21.6,12.7,9.0,189.0,0.02,0.20
1,0.0,2017-10-07 11:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
2,0.0,2017-10-07 12:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,2.7,2017-10-07 13:00:00,2.19,0.20,67.0,28.2,13.0,8.0,149.0,0.03,0.13
4,0.0,2017-10-07 14:00:00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00


In [7]:
data = data.rename(columns={'DateTime_': 'Fecha'})


In [8]:
# Normalización de los datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Fecha']))

In [9]:
# Definir secuencias de entrada para LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :])  # Incluye variables exógenas
        y.append(data[i+seq_length, 0])  # Predecimos la actividad volcánica
    return np.array(X), np.array(y)

seq_length = 48  # Ventana de tiempo de 4 días
X, y = create_sequences(data_scaled, seq_length)

In [10]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [11]:
# Construcción del modelo LSTM Bidireccional
n_features = data.shape[1] - 1  # Número de features, excluyendo la columna objetivo

# Asegúrate de que input_shape coincida con la forma de tus datos de entrada
model = Sequential([
    Bidirectional(LSTM(50, return_sequences=True, activation='relu'), input_shape=(seq_length, n_features)),  # Corrección aquí
    Dropout(0.2),
    #Bidirectional(LSTM(50, return_sequences=True, activation='relu')),
    #Dropout(0.2),
    Bidirectional(LSTM(50, activation='relu')),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# Compilación del modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [13]:
# Entrenamiento del modelo
#model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 86s 82ms/step - loss: 0.0238 - mae: 0.0536 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 2/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 80s 80ms/step - loss: 0.0011 - mae: 0.0030 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 3/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - loss: 9.5419e-04 - mae: 0.0027 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 4/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 137s 81ms/step - loss: 9.4654e-04 - mae: 0.0027 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 5/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 84s 83ms/step - loss: 8.5109e-04 - mae: 0.0024 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 6/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - loss: 0.0010 - mae: 0.0028 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 7/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 83s 87ms/step - loss: 9.0390e-04 - mae: 0.0026 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 8/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 138s 83ms/step - loss: 7.8199e-04 - mae: 0.0022 - val_loss: 0.0958 - val_mae: 0.1610
Epoch 9/20
951/951

In [14]:
def predict_for_dates(model, df, scaler, start_date, days_to_predict):
    """ Predice la actividad volcánica para una serie de fechas futuras. """
    # Convert 'Fecha' column to datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])

    # Convert start_date to Timestamp before comparison
    start_date = pd.to_datetime(start_date)
    input_data = df[df['Fecha'] >= start_date].drop(columns=['Fecha']).values
    input_scaled = scaler.transform(input_data)[-seq_length:]  # Últimos seq_length datos

    predictions = []
    current_input = np.array([input_scaled])  # Reshape para que coincida con el modelo

    for _ in range(days_to_predict):
        pred = model.predict(current_input)[0][0]
        predictions.append(pred)

        # Desplazar ventana de tiempo agregando la nueva predicción (usamos 0s en exógenas como ejemplo)
        new_input = np.hstack([[pred], np.zeros(n_features)])
        current_input = np.append(current_input[:, 1:, :], [[new_input]], axis=1)

    return predictions

In [15]:
# Predicción para fechas específicas
def predict_for_dates(model, df, scaler, start_date, days_to_predict):
    """ Predice la actividad volcánica para una serie de fechas futuras. """
    # Convert 'Fecha' column to datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])

    # Convert start_date to Timestamp before comparison
    start_date = pd.to_datetime(start_date)
    input_data = df[df['Fecha'] >= start_date].drop(columns=['Fecha']).values
    input_scaled = scaler.transform(input_data)[-seq_length:]  # Últimos seq_length datos

    predictions = []
    current_input = np.array([input_scaled])  # Reshape para que coincida con el modelo

    for _ in range(days_to_predict):
        pred = model.predict(current_input)[0][0]
        predictions.append(pred)

        # Desplazar ventana de tiempo agregando la nueva predicción
        # Reemplaza los valores anteriores en lugar de usar append
        # Crea un nuevo input con la predicción y ceros para las otras features
        new_input = np.hstack([[pred], np.zeros(n_features -1 )]) #n_features -1

        # Actualiza current_input desplazando los valores y agregando la nueva entrada
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1, :] = new_input

    return predictions

In [16]:
# Definir fechas para predecir
start_date = data['Fecha'].max()
print(f"La última fecha en el conjunto de datos es: {start_date}")
days_to_predict = 12

La última fecha en el conjunto de datos es: 2022-02-10 06:00:00


In [17]:
# Obtener predicciones
future_predictions = predict_for_dates(model, data, scaler, start_date, days_to_predict)

# Mostrar resultados con fechas
future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="1H")
predictions_df = pd.DataFrame({"Fecha": future_dates, "Predicción Actividad": future_predictions})

print(predictions_df)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
                 Fecha  Predicción Actividad
0  2022-02-10 06:00:00              0.448613
1  2022-02-10 07:00:00              0.458938
2  2022-02-10 08:00:00              0.458754
3  2022-02-10 09:00:00              0.458757
4  2022-02-10 10:00:00              0.458757
5  2022-02-10 11:00:00              0.458757
6  2022-02-10 12:00:00              0.458757
7  2022-02-10 13:00:00              0.458757
8  2022-02-10 14:00:00              0.458757
9  2022-02-10 15:00:00              0.458757
10 2022-02-10 16:00:00              0.458757
11 

<ipython-input-17-fe85a30d84ec>:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="1H")


In [18]:
# Predicción
predictions = model.predict(X_test)

# Mostrar una predicción de ejemplo
print(f"Predicción de actividad volcánica: {predictions[0][0]}")

238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step
Predicción de actividad volcánica: 0.0


In [19]:
# Crear un DataFrame con los primeros valores reales y predichos
df_comparacion = pd.DataFrame({"Real": y_test.flatten(), "Predicho": predictions.flatten()})

# Mostrar las primeras filas
print(df_comparacion.head(10))  # Puedes cambiar 10 por otro número

   Real  Predicho
0   0.0       0.0
1   0.0       0.0
2   0.0       0.0
3   0.0       0.0
4   0.0       0.0
5   0.0       0.0
6   0.0       0.0
7   0.0       0.0
8   0.0       0.0
9   0.0       0.0


In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calcular métricas de error
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)  # Raíz del error cuadrático medio
r2 = r2_score(y_test, predictions)  # Coeficiente de determinación R^2

# Mostrar los resultados
print(f"MAE (Error Absoluto Medio): {mae:.4f}")
print(f"MSE (Error Cuadrático Medio): {mse:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R^2 (Coeficiente de Determinación): {r2:.4f}")


MAE (Error Absoluto Medio): 0.1610
MSE (Error Cuadrático Medio): 0.0958
RMSE (Raíz del Error Cuadrático Medio): 0.3095
R^2 (Coeficiente de Determinación): -0.3708


###RNN y GRU

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [23]:

# Cargar datos
data = pd.read_csv("Datos_Con1HResampled.csv")
data = data.rename(columns={'DateTime_': 'Fecha'})
data['Fecha'] = pd.to_datetime(data['Fecha'])

In [24]:
# Mover la columna "Magnitud" a la posición 0
magnitud_column = data['Magnitud']
data = data.drop('Magnitud', axis=1)
data.insert(0, 'Magnitud', magnitud_column)

In [25]:
# Normalización de los datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Fecha']))

In [26]:
# Función para crear secuencias de entrada
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :])
        y.append(data[i+seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 48  # Ventana de tiempo de 4 días
X, y = create_sequences(data_scaled, seq_length)

In [27]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

n_features = data.shape[1] - 1  # Número de características (excluyendo 'Magnitud')

In [28]:

# Modelo RNN
model_rnn = Sequential([
    SimpleRNN(50, return_sequences=True, activation='relu', input_shape=(seq_length, n_features)),
    Dropout(0.2),
    #SimpleRNN(50, return_sequences=True, activation='relu'),
    #Dropout(0.2),
    SimpleRNN(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
# Modelo GRU
model_gru = Sequential([
    GRU(50, return_sequences=True, activation='relu', input_shape=(seq_length, n_features)),
    Dropout(0.2),
    #GRU(50, return_sequences=True, activation='relu'),
    #Dropout(0.2),
    GRU(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

In [30]:
# Compilar modelos



































for model, name in [(model_rnn, "RNN"), (model_gru, "GRU")]:
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print(f"Entrenando modelo {name}...")
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Entrenando modelo RNN...
Epoch 1/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 29s 27ms/step - loss: 9.6972e-04 - mae: 0.0059 - val_loss: 0.0577 - val_mae: 0.1316
Epoch 2/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - loss: 7.3049e-04 - mae: 0.0043 - val_loss: 0.0732 - val_mae: 0.1449
Epoch 3/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - loss: 8.6370e-04 - mae: 0.0053 - val_loss: 0.0673 - val_mae: 0.1409
Epoch 4/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - loss: 7.3706e-04 - mae: 0.0047 - val_loss: 0.0502 - val_mae: 0.1252
Epoch 5/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - loss: 7.6974e-04 - mae: 0.0049 - val_loss: 0.0639 - val_mae: 0.1367
Epoch 6/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 42s 24ms/step - loss: 7.1155e-04 - mae: 0.0043 - val_loss: 0.0732 - val_mae: 0.1456
Epoch 7/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 43s 26ms/step - loss: 6.7692e-04 - mae: 0.0042 - val_loss: 0.0487 - val_mae: 0.1239
Epoch 8/20
951/951 ━━━━━━━━━━━━━━━━━━━━ 38s 22ms/step - loss: 7.5788e-04 - mae: 0.0045 - val_loss: 0.0670 -

In [31]:
# Función para predecir fechas futuras
def predict_for_dates(model, df, scaler, start_date, days_to_predict):
    start_date = pd.to_datetime(start_date)
    df['Fecha'] = pd.to_datetime(df['Fecha'])

    input_data = df[df['Fecha'] >= start_date].drop(columns=['Fecha']).values
    input_scaled = scaler.transform(input_data)[-seq_length:]

    predictions = []
    current_input = np.array([input_scaled])

    for _ in range(days_to_predict):
        pred = model.predict(current_input)[0][0]
        predictions.append(pred)

        new_input = np.hstack([[pred], np.zeros(n_features - 1)])
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1, :] = new_input

    return predictions

In [32]:
# Definir fechas para predecir
start_date = data['Fecha'].max()
print(f"La última fecha en el conjunto de datos es: {start_date}")
days_to_predict = 12
future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="1H")

La última fecha en el conjunto de datos es: 2022-02-10 06:00:00


<ipython-input-32-cf83d07b427c>:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="1H")


In [33]:
# Obtener predicciones para ambos modelos
future_predictions_rnn = predict_for_dates(model_rnn, data, scaler, start_date, days_to_predict)
future_predictions_gru = predict_for_dates(model_gru, data, scaler, start_date, days_to_predict)

# Crear DataFrame con predicciones
predictions_df = pd.DataFrame({
    "Fecha": future_dates,
    "Predicción RNN": future_predictions_rnn,
    "Predicción GRU": future_predictions_gru
})

print(predictions_df)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
                 Fecha  Predicción RNN  Predicción GRU
0  2022-02-10 06:00:00        0.144825        0.048981
1  2022-02-10 07:00:00        0.015008        0.001169
2  2022-02-10 08:00:00        0.003662        0.001168
3  2022-02-10 09:00:00        0.003100        0.001168
4  2022-02-10 10:00:00        0.003082        0.001168
5  2022-02-10 11:00:00        0.003081        0.001168
6  2022-02-10 12:00:00        0.003081        0.001168
7  2022-02-10 13:00:00        0.003081        0.001168
8  2022-02-10 14:00:00        0.003081        0.

In [34]:
# Evaluar modelos con métricas
for model, name in [(model_rnn, "RNN"), (model_gru, "GRU")]:
    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    print(f"\nMétricas para {name}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

Métricas para RNN:
MAE: 0.1281
MSE: 0.0545
RMSE: 0.2334
R²: 0.2202
238/238 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step

Métricas para GRU:
MAE: 0.1250
MSE: 0.0526
RMSE: 0.2293
R²: 0.2473
